<a href="https://colab.research.google.com/github/FacuRoffet99/ML/blob/main/TP6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicio 4: IMDB Rating

In [1]:
import pandas as pd
import numpy as np

# Carga y preprocesamiento de datos

In [2]:
# Cargar dataset
imdb_data = pd.read_csv("https://raw.githubusercontent.com/emmanueliarussi/DataScienceCapstone/master/3_MidtermProjects/ProjectIMDB/data/movie_metadata.csv")
imdb_data.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,actor_1_name,movie_title,num_voted_users,cast_total_facebook_likes,actor_3_name,facenumber_in_poster,plot_keywords,movie_imdb_link,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,CCH Pounder,Avatar,886204,4834,Wes Studi,0.0,avatar|future|marine|native|paraplegic,http://www.imdb.com/title/tt0499549/?ref_=fn_t...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,Johnny Depp,Pirates of the Caribbean: At World's End,471220,48350,Jack Davenport,0.0,goddess|marriage ceremony|marriage proposal|pi...,http://www.imdb.com/title/tt0449088/?ref_=fn_t...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,Christoph Waltz,Spectre,275868,11700,Stephanie Sigman,1.0,bomb|espionage|sequel|spy|terrorist,http://www.imdb.com/title/tt2379713/?ref_=fn_t...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,Tom Hardy,The Dark Knight Rises,1144337,106759,Joseph Gordon-Levitt,0.0,deception|imprisonment|lawlessness|police offi...,http://www.imdb.com/title/tt1345836/?ref_=fn_t...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,Doug Walker,Star Wars: Episode VII - The Force Awakens ...,8,143,NaN,0.0,NaN,http://www.imdb.com/title/tt5289954/?ref_=fn_t...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


In [3]:
# Eliminar las features que no puedo codificar
imdb_data.drop(["director_name","actor_2_name","genres","movie_title","actor_1_name","actor_3_name","plot_keywords","movie_imdb_link"],axis=1, inplace = True)
imdb_data.head()

,color,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_1_facebook_likes,gross,num_voted_users,cast_total_facebook_likes,facenumber_in_poster,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,723.0,178.0,0.0,855.0,1000.0,760505847.0,886204,4834,0.0,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,302.0,169.0,563.0,1000.0,40000.0,309404152.0,471220,48350,0.0,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,602.0,148.0,0.0,161.0,11000.0,200074175.0,275868,11700,1.0,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,813.0,164.0,22000.0,23000.0,27000.0,448130642.0,1144337,106759,0.0,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,NaN,NaN,131.0,NaN,131.0,NaN,8,143,0.0,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


In [4]:
# Ver si hay valores nulos: hay unos cuantos
imdb_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5043 entries, 0 to 5042
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   color                      5024 non-null   object 
 1   num_critic_for_reviews     4993 non-null   float64
 2   duration                   5028 non-null   float64
 3   director_facebook_likes    4939 non-null   float64
 4   actor_3_facebook_likes     5020 non-null   float64
 5   actor_1_facebook_likes     5036 non-null   float64
 6   gross                      4159 non-null   float64
 7   num_voted_users            5043 non-null   int64  
 8   cast_total_facebook_likes  5043 non-null   int64  
 9   facenumber_in_poster       5030 non-null   float64
 10  num_user_for_reviews       5022 non-null   float64
 11  language                   5031 non-null   object 
 12  country                    5038 non-null   object 
 13  content_rating             4740 non-null   objec

In [5]:
# Rellenar los valores numéricos nulos con una interpolación
imdb_data = imdb_data.interpolate()
imdb_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5043 entries, 0 to 5042
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   color                      5024 non-null   object 
 1   num_critic_for_reviews     5043 non-null   float64
 2   duration                   5043 non-null   float64
 3   director_facebook_likes    5043 non-null   float64
 4   actor_3_facebook_likes     5043 non-null   float64
 5   actor_1_facebook_likes     5043 non-null   float64
 6   gross                      5043 non-null   float64
 7   num_voted_users            5043 non-null   int64  
 8   cast_total_facebook_likes  5043 non-null   int64  
 9   facenumber_in_poster       5043 non-null   float64
 10  num_user_for_reviews       5043 non-null   float64
 11  language                   5031 non-null   object 
 12  country                    5038 non-null   object 
 13  content_rating             4740 non-null   objec

In [6]:
# Eliminar los datos que todavía tienen valores nulos
imdb_data = imdb_data.dropna()
imdb_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4720 entries, 0 to 5042
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   color                      4720 non-null   object 
 1   num_critic_for_reviews     4720 non-null   float64
 2   duration                   4720 non-null   float64
 3   director_facebook_likes    4720 non-null   float64
 4   actor_3_facebook_likes     4720 non-null   float64
 5   actor_1_facebook_likes     4720 non-null   float64
 6   gross                      4720 non-null   float64
 7   num_voted_users            4720 non-null   int64  
 8   cast_total_facebook_likes  4720 non-null   int64  
 9   facenumber_in_poster       4720 non-null   float64
 10  num_user_for_reviews       4720 non-null   float64
 11  language                   4720 non-null   object 
 12  country                    4720 non-null   object 
 13  content_rating             4720 non-null   objec

In [7]:
# Convertir las variables categóricas a numéricas
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

imdb_data["new_color"] = labelencoder.fit_transform(imdb_data["color"])
imdb_data["new_country"] = labelencoder.fit_transform(imdb_data["country"])
imdb_data["new_language"] = labelencoder.fit_transform(imdb_data["language"])
imdb_data["new_content_rating"] = labelencoder.fit_transform(imdb_data["content_rating"])

imdb_data.drop(["color","country","language","content_rating"],axis=1, inplace = True)
imdb_data.head()

,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_1_facebook_likes,gross,num_voted_users,cast_total_facebook_likes,facenumber_in_poster,num_user_for_reviews,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes,new_color,new_country,new_language,new_content_rating
0,723.0,178.0,0.0,855.0,1000.0,760505847.0,886204,4834,0.0,3054.0,237000000.0,2009.0,936.0,7.9,1.78,33000,1,56,10,7
1,302.0,169.0,563.0,1000.0,40000.0,309404152.0,471220,48350,0.0,1238.0,300000000.0,2007.0,5000.0,7.1,2.35,0,1,56,10,7
2,602.0,148.0,0.0,161.0,11000.0,200074175.0,275868,11700,1.0,994.0,245000000.0,2015.0,393.0,6.8,2.35,85000,1,55,10,7
3,813.0,164.0,22000.0,23000.0,27000.0,448130642.0,1144337,106759,0.0,2701.0,250000000.0,2012.0,23000.0,8.5,2.35,164000,1,56,10,7
5,462.0,132.0,475.0,530.0,640.0,73058679.0,212204,1873,1.0,738.0,263700000.0,2012.0,632.0,6.6,2.35,24000,1,56,10,7


In [8]:
# Dividir en datos y etiquetas
target = imdb_data["imdb_score"]
data = imdb_data.drop(["imdb_score"],axis=1)

# Escalar los datos
from sklearn.preprocessing import scale
data = pd.DataFrame(scale(data), columns=data.columns)

# Dividir en train y test
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data, target, test_size = 0.2)

# Regresión lineal
La regresión lineal es un buen punto de partida para comparar a los demás modelos.

In [18]:
# Entrenar el modelo
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(x_train,y_train)

# Predicciones
y_pred = model.predict(x_test)

# Métricas
from sklearn import metrics
print("R2 =",model.score(x_test,y_test))
print("MAE =", metrics.mean_absolute_error(y_test, y_pred))
print("MSE =",metrics.mean_squared_error(y_test, y_pred))
print("RMSE =",np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

R2 = 0.3591222009902112
MAE = 0.6935666334215934
MSE = 0.8310979782669722
RMSE = 0.9116457526182921


# Regresión lineal reducida
Probando con solamente las features más correlacionadas, se entrena un nuevo modelo. Los resultados son muy parecidos, pero levemente peores.

In [10]:
# Miro los coeficientes del modelo
model.coef_

array([ 0.30437918,  0.15600319,  0.0220167 ,  0.14735789,  1.15057964,
       -0.09361301,  0.50708492, -1.38164277, -0.04677655, -0.22114295,
       -0.01134694, -0.30960903,  0.33141173,  0.11136602, -0.01254998,
       -0.06929427, -0.0471667 ,  0.09448013,  0.07679878])

In [11]:
# Me quedo con las features con abs(coef)>0.1
reduced_features = ['num_critic_for_reviews',	'duration', 'actor_1_facebook_likes',
                    'num_voted_users', 'cast_total_facebook_likes', 'num_user_for_reviews',
                    'title_year', 'actor_2_facebook_likes',	'aspect_ratio',
                    'new_language']
x_train_reduced = x_train[reduced_features]
y_train_reduced = y_train
x_test_reduced = x_test[reduced_features]
y_test_reduced = y_test

# Entreno un nuevo modelo
model_reduced = LinearRegression()
model_reduced.fit(x_train_reduced,y_train_reduced)

# Predicciones
y_pred_reduced = model_reduced.predict(x_test_reduced)

# Métricas
print("R2 =",model_reduced.score(x_test_reduced,y_test_reduced))
print("MAE =", metrics.mean_absolute_error(y_test_reduced, y_pred_reduced))
print("MSE =",metrics.mean_squared_error(y_test_reduced, y_pred_reduced))
print("RMSE =",np.sqrt(metrics.mean_squared_error(y_test_reduced, y_pred_reduced)))

R2 = 0.33875545950303243
MAE = 0.7049626461511249
MSE = 0.8575098116929912
RMSE = 0.9260182566736961


# Regresión Ridge
Este modelo utiliza distintos valores de regularización, logrando una muy pequeña mejoría por sobre la regresión lineal.

In [12]:
# Entrenar el modelo
from sklearn.linear_model import RidgeCV
alphas = np.linspace(0.1,10,100)
model = RidgeCV(alphas=alphas, cv=4)
model.fit(x_train,y_train)
# Predicciones
y_pred = model.predict(x_test)

# Métricas
from sklearn import metrics
print("R2 =",model.score(x_test,y_test))
print("MAE =", metrics.mean_absolute_error(y_test, y_pred))
print("MSE =",metrics.mean_squared_error(y_test, y_pred))
print("RMSE =",np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

R2 = 0.3601901930341005
MAE = 0.693102848906676
MSE = 0.8297129934385805
RMSE = 0.9108858289810972


# Regresión Ridge reducida
Nuevamente utilizando menos features, el resultado empeora.

In [13]:
# Miro los coeficientes del modelo
model.coef_

array([ 0.3045886 ,  0.15585101,  0.02192431,  0.13154345,  1.0640019 ,
       -0.09495928,  0.50687918, -1.27550622, -0.04674105, -0.2207849 ,
       -0.01136165, -0.3098029 ,  0.30645759,  0.11119676, -0.01258016,
       -0.06931594, -0.0477516 ,  0.09468395,  0.07675182])

In [14]:
# Me quedo con las features con abs(coef)>0.1
reduced_features = ['num_critic_for_reviews',	'duration', 'actor_1_facebook_likes',
                    'num_voted_users', 'cast_total_facebook_likes', 'num_user_for_reviews',
                    'title_year', 'actor_2_facebook_likes',	'aspect_ratio',
                    'new_language']
x_train_reduced = x_train[reduced_features]
y_train_reduced = y_train
x_test_reduced = x_test[reduced_features]
y_test_reduced = y_test

# Entreno un nuevo modelo
model_reduced = RidgeCV(alphas=alphas, cv=4)
model_reduced.fit(x_train_reduced,y_train_reduced)

# Predicciones
y_pred_reduced = model_reduced.predict(x_test_reduced)

# Métricas
print("R2 =",model_reduced.score(x_test_reduced,y_test_reduced))
print("MAE =", metrics.mean_absolute_error(y_test_reduced, y_pred_reduced))
print("MSE =",metrics.mean_squared_error(y_test_reduced, y_pred_reduced))
print("RMSE =",np.sqrt(metrics.mean_squared_error(y_test_reduced, y_pred_reduced)))

R2 = 0.33934552085189806
MAE = 0.7046030440581027
MSE = 0.8567446130937367
RMSE = 0.9256049984165691


# Regresión Lasso
Con este modelo, se volvió a obtener una pequeña mejora en las métricas.

In [23]:
# Entrenar el modelo
from sklearn.linear_model import LassoCV
model = LassoCV(cv=4, max_iter=10000)
model.fit(x_train,y_train)
# Predicciones
y_pred = model.predict(x_test)

# Métricas
from sklearn import metrics
print("R2 =",model.score(x_test,y_test))
print("MAE =", metrics.mean_absolute_error(y_test, y_pred))
print("MSE =",metrics.mean_squared_error(y_test, y_pred))
print("RMSE =",np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

R2 = 0.3617816119477414
MAE = 0.6923578211917663
MSE = 0.8276492223986937
RMSE = 0.9097522862838509


# Regresión Lasso reducida
De nuevo, utilizar menos features no funciona.

In [16]:
# Miro los coeficientes del modelo
model.coef_

array([ 0.3031029 ,  0.15502234,  0.02142142,  0.09657137,  0.87675324,
       -0.09708234,  0.5044356 , -1.04573463, -0.04612094, -0.21772832,
       -0.0109283 , -0.30975371,  0.25226349,  0.11020669, -0.0108206 ,
       -0.06907628, -0.04864533,  0.09484793,  0.07631645])

In [24]:
# Me quedo con las features con abs(coef)>0.1
reduced_features = ['num_critic_for_reviews',	'duration', 'actor_1_facebook_likes',
                    'num_voted_users', 'cast_total_facebook_likes', 'num_user_for_reviews',
                    'title_year', 'actor_2_facebook_likes',	'aspect_ratio',
                    'new_language']
x_train_reduced = x_train[reduced_features]
y_train_reduced = y_train
x_test_reduced = x_test[reduced_features]
y_test_reduced = y_test

# Entreno un nuevo modelo
model_reduced = LassoCV(cv=4, max_iter=10000)
model_reduced.fit(x_train_reduced,y_train_reduced)

# Predicciones
y_pred_reduced = model_reduced.predict(x_test_reduced)

# Métricas
print("R2 =",model_reduced.score(x_test_reduced,y_test_reduced))
print("MAE =", metrics.mean_absolute_error(y_test_reduced, y_pred_reduced))
print("MSE =",metrics.mean_squared_error(y_test_reduced, y_pred_reduced))
print("RMSE =",np.sqrt(metrics.mean_squared_error(y_test_reduced, y_pred_reduced)))

R2 = 0.33914003804900283
MAE = 0.7047043201730178
MSE = 0.8570110856448513
RMSE = 0.9257489322947401


# Conclusiones
Todos los modelos utilizados arrojaron resultados similares, siendo Lasso el de mejor performance. La técnica de eliminación de features poco correlacionadas no fue util en este caso, aunque podrían hacerse más pruebas al respecto. 